In [1]:
## Parsing csv-data for use with TensorFlow
import parse_csv

In [2]:
# Parse Wine-data

src = "../DATA/wine/wine.data.txt"
labels = ('1', '2', '3')  # note: always str list!

# In wine-data y is first column, so y_last=False
wine_data = parse_csv.ParseCsv(src, labels, y_last=False)

# Get a shuffled batch with all instances, targets hot-one, normalized, and ratio of 10:
wine_x_train, wine_y_train, wine_x_test, wine_y_test = wine_data.getBatch(None, normalize=True)

### To change defaults use arguments:
#   n_inst=num_you_want (mandatory, use None if nothing else)
#   shuffle=False  (you should never do this, though)
#   hotone=False
#   normalize=True
#   ratio=num_you_want

## HOX: there should also be a validation set, getBatch needs to be rethought

In [3]:
## Testing with TF-softmax

import tensorflow as tf
import numpy as np

In [4]:
## With wine data
#  Set up model

wine_nx = len(wine_data.data[0])-1
wine_ny = len(wine_data.labels)

# input values
x = tf.placeholder(tf.float32, [None, wine_nx])

# weights and biases
W = tf.Variable(tf.truncated_normal([wine_nx, wine_ny], stddev=0.01))
b = tf.Variable(tf.truncated_normal([wine_ny], stddev=0.01))

# model
mmul = tf.matmul(x, W) + b
y = tf.nn.softmax(mmul)

# target
y_ = tf.placeholder(tf.float32, [None, wine_ny])

# error  --- here's a danger of division by 0 !
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0)))

# training step with GD
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)


In [5]:
## Init & run session

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# evaluation criteria
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# train and test

for i in range(30):
    for j in range(1000):
        sess.run(train_step, feed_dict={x: wine_x_train, y_: wine_y_train})
    acc = sess.run(accuracy, feed_dict={x: wine_x_test, y_: wine_y_test})
    print(i, acc)
    if acc >= 0.99:
        break
    # Uncomment to see weight & bias matrixes during training:
    # print('W: ', W.eval(sess))
    # print('b: ', b.eval(sess))


W:  [ 1.23261452 -1.47770643  0.2330769 ]
0 0.941176
W:  [ 1.28244162 -1.5388037   0.24434698]
b:  [-0.01711994  0.27079725 -0.26261213]
1 0.941176
W:  [ 1.33133638 -1.59885597  0.25550395]
b:  [-0.01782992  0.27421069 -0.26531413]
2 0.941176
W:  [ 1.37932456 -1.65788579  0.26654622]
b:  [-0.01850324  0.27760991 -0.2680518 ]
3 0.941176
W:  [ 1.42643631 -1.71592367  0.27747285]
b:  [-0.01913877  0.28100738 -0.27082223]
4 0.941176
W:  [ 1.47269547 -1.77299333  0.28828347]
b:  [-0.01973326  0.28438768 -0.27361241]
5 0.941176
W:  [ 1.51812696 -1.8291198   0.29897767]
b:  [-0.02028979  0.28774902 -0.27641383]
6 0.941176
W:  [ 1.56275761 -1.88432968  0.30955529]
b:  [-0.02080911  0.29108465 -0.27924123]
7 0.941176
W:  [ 1.60661221 -1.93864369  0.3200165 ]
b:  [-0.02129315  0.29439172 -0.28207245]
8 0.941176
W:  [ 1.64971256 -1.99208868  0.33036157]
b:  [-0.02174123  0.29766959 -0.28490368]
9 0.941176
W:  [ 1.69207978 -2.04468536  0.34059113]
b:  [-0.02215351  0.30091676 -0.2877349 ]


In [ ]:
## Note: the model might be overfitting. Separate validation and test sets needed.